# Personal Finance Library
snippets of code that I made to handle financial data, from pulling data using an API to computing new values

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np

def adjust(date, close, adj_close, in_col, rounding=4):
    '''
    If using forex or Crypto - Change the rounding accordingly!
    '''
    try:
        factor = adj_close / close
        return round(in_col * factor, rounding)
    except ZeroDivisionError:
        print('WARNING: DIRTY DATA >> {} Close: {} | Adj Close {} | in_col: {}'.format(date, close, adj_close, in_col))
        return 0

def getTickerData(ticker, start,end):
    '''
    funciton to fetch data of a stock (ticker) and compute the daily returns, log returns, cumulative returns usind split adjusted data
    
    Parameters:
    -----------  
     ticker: symbol of the stock to fetch data (US)
      start: start date in ISO format yyyy-mm-dd
        end: end date in ISO format yyyy-mm-dd
            
    Returns:
    --------
        df: with the following columns
            'adj open','adj close','adj high', 'adj low', 'daily_return', 'cumluative_return' , 'log_return'
    '''
    df = yf.download(ticker,start_date,end_date)
    df['Adj Open'] = np.vectorize(adjust)(df.index.date, df['Close'], df['Adj Close'], df['Open'])
    df['Adj High'] = np.vectorize(adjust)(df.index.date, df['Close'], df['Adj Close'], df['High'])
    df['Adj Low'] = np.vectorize(adjust)(df.index.date, df['Close'], df['Adj Close'], df['Low'])    
    df['Daily Return'] = df['Adj Close'].pct_change()
    df['Log Return'] = np.log(df['Adj Close']/df['Adj Close'].shift(1)) #np.log(1+df['adj close'].pct_change())
    df['Cumulative Return'] = np.exp(np.log1p(df['Daily Return']).cumsum())
    
    return df


def getPtf (tickers, start, end):
    '''
    funciton to fetch data of a series of tickers and build a portfolio
    
    Parameters:
    -----------  
     tickers: list of tickers for which to fetch data
            
    Returns:
    --------
     ptf: multi index data frame with stock data divided on the columns
       
    '''
    ptf = pd.DataFrame()
    df = pd.DataFrame()
    for stock in tickers:
        df = getTickerData(stock, start, end)
        df.sort_index(ascending=True, inplace = True)
        df.columns = pd.MultiIndex.from_product([[stock], df.columns])
        ptf =  pd.concat([df, ptf], axis=1)
    return ptf

def randomWeights(n):
    '''
    funciton to retrun a np array of n random numbers, wich all add up to 1
    
    Parameters:
    -----------  
     n: number of assets
            
    Returns:
    --------
     list: np array 
       
    '''
    k = np.random.rand(n)
    return k/sum(k)


def getSpecificColumns(ptf, column, stocks):
    '''
    funciton to get the specific columns for each stock contained in the ptf
    
    Parameters:
    -----------  
     ptf: ptf obtained with getPtfColumns 
     column: name of the column
     stocks: list of the tickers    
            
    Returns:
    --------
     new_ptf: dataframe of the specific columns for each stock
       
    '''
    df = pd.DataFrame()
    new_ptf = pd.DataFrame()
    for each in stocks:
        df = ptf.loc[:, each][[column]]
        df.columns = [each]
        new_ptf = pd.concat([df, new_ptf], axis=1)
    return new_ptf

def ptfPerformance(weights, ptf):
    '''
    funciton to return a tuple of ptf return and std dev annualised, function accepts df from getPtfColumns, and isolates the 
    returns and computes std dev
    
    Parameters:
    -----------  
     weight: weight of the assets
     ptf: dataframe of stock ptf  
            
    Returns:
    --------
     new_ptf: dataframe of the specific columns for each stock
       
    '''    
    returns = getSpecificColumns(ptf, 'Daily Return', ptf.columns.levels[0])
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    weights_a = np.array(weights)
    ptf_return = np.sum(mean_returns*weights_a)*252
    ptf_std = np.sqrt(np.dot(weights_a.T, np.dot(cov_matrix, weights_a ))) * np.sqrt(252)
    return ptf_return, ptf_std

def retrurnsAndStd(ptf, stocks, iterations):
    '''
           
    '''    
    df = pd.DataFrame(columns = ['return', 'std', 'weights'])
    for i in range(iterations):
        weights = randomWeights(len(stocks))
        returns, std = ptfPerformance(weights, ptf)
        df.loc[i] = returns, std, weights
    return df

def stockPerformanceAnnualised(stock_daily_return):
    '''
    function to annualise the returns of a stock
    
    Parameter:
    -----------
        stock_daily_return:
    
    Returns:
    -----------
        
    ''' 
    return np.mean(stock_daily_return)*252, np.std(stock_daily_return)*np.sqrt(252)